In [6]:
# import h2o package and specific estimator 
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

import warnings
warnings.filterwarnings('ignore')

In [5]:
h2o.init(strict_version_check=False) # start h2o

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,2 mins 31 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_newzysharma_p0cdw3
H2O cluster total nodes:,1
H2O cluster free memory:,2 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [12]:
#importing data to the server
hp = h2o.import_file(path="hour.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
#Displaying the head
hp.head()

instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2011-01-01 00:00:00,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
2,2011-01-01 00:00:00,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0,8,32,40
3,2011-01-01 00:00:00,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0,5,27,32
4,2011-01-01 00:00:00,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13
5,2011-01-01 00:00:00,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0,0,1,1
6,2011-01-01 00:00:00,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
7,2011-01-01 00:00:00,1,0,1,6,0,6,0,1,0.22,0.2727,0.8,0,2,0,2
8,2011-01-01 00:00:00,1,0,1,7,0,6,0,1,0.2,0.2576,0.86,0,1,2,3
9,2011-01-01 00:00:00,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7,8
10,2011-01-01 00:00:00,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0,8,6,14


In [14]:
hp.describe()

Rows:17379
Cols:17




,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
type,int,time,int,int,int,int,int,int,int,int,real,real,real,real,int,int,int
mins,1.0,1293840000000.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.02,0.0,0.0,0.0,0.0,0.0,1.0
mean,8690.0,1325477314552.0461,2.501639910236492,0.5025605615973301,6.5377754761493785,11.546751826917548,0.028770355026181024,3.003682605443351,0.6827205247712756,1.425283387997008,0.4969871684216584,0.47577510213476026,0.6272288394038784,0.1900976063064618,35.67621842453536,153.78686920996606,189.4630876345014
maxs,17379.0,1356912000000.0,4.0,1.0,12.0,23.0,1.0,6.0,1.0,4.0,1.0,1.0,1.0,0.8507,367.0,886.0,977.0
sigma,5017.029499614288,18150225217.779854,1.1069181394480765,0.5000078290910197,3.438775713750168,6.914405095264493,0.16716527638437123,2.005771456110988,0.4654306335238829,0.6393568777542534,0.19255612124972193,0.17185021563535932,0.19292983406291514,0.1223402285727905,49.30503038705309,151.35728591258314,181.38759909186476
zeros,0,0,0,8645,0,726,16879,2502,5514,0,0,2,22,2180,1581,24,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,2011-01-01 00:00:00,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,3.0,13.0,16.0
1,2.0,2011-01-01 00:00:00,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,8.0,32.0,40.0
2,3.0,2011-01-01 00:00:00,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,5.0,27.0,32.0


# Model with 500 seconds

In [37]:
# Assume the following are passed by the user from the web interface

'''
Need a user id and project id?

'''
target='cnt' 
data_file='hour.csv'
run_time=500
run_id='SOME_ID_20180617_221528' # Just some arbitrary ID
server_path='Users/newzysharma/Desktop/Desktop/Machine_Learning/Project'
classification=True
scale=False
max_models=None
balance_y=False # balance_classes=balance_y
balance_threshold=0.2
project ="HyperparameterDB_Project"  # project_name = project

In [42]:
# assign target and inputs for logistic regression
y = target
X = [name for name in df.columns if name != y]
print(y)
print(X)

cnt
['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered']


In [43]:
# determine column types
ints, reals, enums = [], [], []
for key, val in df.types.items():
    if key in X:
        if val == 'enum':
            enums.append(key)
        elif val == 'int':
            ints.append(key)            
        else: 
            reals.append(key)

print(ints)
print(enums)
print(reals)

['instant', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'casual', 'registered']
[]
['dteday', 'temp', 'atemp', 'hum', 'windspeed']


In [44]:
# impute missing values
_ = df[reals].impute(method='mean')
_ = df[ints].impute(method='median')

if scale:
    df[reals] = df[reals].scale()
    df[ints] = df[ints].scale()

In [45]:
# set target to factor for classification by default or if user specifies classification
if classification:
    df[y] = df[y].asfactor()

In [46]:
df[y].levels()

[['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '79',
  '80',
  '81',
  '82',
  '83',
  '84',
  '85',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '100',
  '101',
  '102',
  '103',
  '104',
  '105',
  '106',
  '107',
  '108',
  '109',
  '110',
  '111',
  '112',
  '113',
  '114',
  '115',
  '116',
  '117',
  '118',
  '119',
  '120',
  '121',
  '122',
  '123',
 


## Cross-validate rather than take a test training split with 500 seconds

In [47]:
# automl
# runs for run_time seconds then builds a stacked ensemble

aml = H2OAutoML(max_runtime_secs=run_time,project_name = project) # init automl, run for 500 seconds
aml.train(x=X,  
           y=y,
           training_frame=hp)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Leaderboard

In [48]:
# view leaderboard
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_3_AutoML_20190407_222223,10.3938,3.22394,10.3938,1.79838,0.0255248
GBM_1_AutoML_20190407_222223,14.531,3.81196,14.531,2.39932,0.0535151
DeepLearning_1_AutoML_20190407_222223,14.694,3.83328,14.694,2.77942,nan
XGBoost_3_AutoML_20190407_222223,16.5115,4.06343,16.5115,2.65759,0.0686023
GBM_2_AutoML_20190407_222223,17.8634,4.22651,17.8634,2.61351,0.0574055
XGBoost_1_AutoML_20190408_000952,19.2132,4.38329,19.2132,2.5899,0.0497179
StackedEnsemble_AllModels_AutoML_20190407_222223,21.623,4.65005,21.623,3.14915,0.18555
XGBoost_3_AutoML_20190408_001901,25.3854,5.03839,25.3854,3.30893,0.09664
XGBoost_2_AutoML_20190408_000952,25.8932,5.08854,25.8932,2.88628,0.0524873
XGBoost_1_AutoML_20190407_222223,28.4128,5.33037,28.4128,3.35929,0.0995271


In [49]:
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_3_AutoML_20190407_222223


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 2.155227478290276
RMSE: 1.4680693029589154
MAE: 0.9893247655273629
RMSLE: 0.01894810024709767
Mean Residual Deviance: 2.155227478290276

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 10.39381683106471
RMSE: 3.2239442971404935
MAE: 1.798379228502425
RMSLE: 0.025524841564696307
Mean Residual Deviance: 10.39381683106471
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,1.7983793,0.0308816,1.8304428,1.7734675,1.7317806,1.8571346,1.7990708
mean_residual_deviance,10.393801,0.5789512,9.861467,9.708306,10.298363,11.979713,10.121156
mse,10.393801,0.5789512,9.861467,9.708306,10.298363,11.979713,10.121156
r2,0.9996840,0.0000176,0.9997007,0.9997072,0.9996852,0.9996366,0.9996907
residual_deviance,10.393801,0.5789512,9.861467,9.708306,10.298363,11.979713,10.121156
rmse,3.2215536,0.0877322,3.1402972,3.1158154,3.2091062,3.4611723,3.1813765
rmsle,0.0254593,0.0012937,0.0273739,0.0244253,0.0229992,0.0247157,0.0277825


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-07 22:38:06,13.517 sec,0.0,181.3823804,142.3998489,32899.5679309
,2019-04-07 22:38:06,13.582 sec,5.0,107.6288213,84.4261234,11583.9631809
,2019-04-07 22:38:06,13.653 sec,10.0,63.8797153,50.0275595,4080.6180310
,2019-04-07 22:38:06,13.722 sec,15.0,38.3081093,29.9323746,1467.5112348
,2019-04-07 22:38:06,13.791 sec,20.0,22.8327372,17.7868675,521.3338901
---,---,---,---,---,---,---
,2019-04-07 22:38:08,15.781 sec,185.0,1.5150515,1.0157186,2.2953812
,2019-04-07 22:38:08,15.834 sec,190.0,1.5003954,1.0083877,2.2511862
,2019-04-07 22:38:08,15.886 sec,195.0,1.4894064,1.0026039,2.2183313
,2019-04-07 22:38:08,15.938 sec,200.0,1.4792240,0.9959397,2.1881035



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,2522182656.0000000,1.0,0.9019286
casual,173936336.0000000,0.0689626,0.0621994
hr,53307732.0000000,0.0211356,0.0190628
workingday,22751658.0000000,0.0090206,0.0081360
temp,10054809.0000000,0.0039866,0.0035956
dteday,6581487.5,0.0026094,0.0023535
atemp,1724350.3750000,0.0006837,0.0006166
weekday,1579544.0,0.0006263,0.0005648
weathersit,1216864.2500000,0.0004825,0.0004351
instant,1135067.1250000,0.0004500,0.0004059


In [56]:
aml.leader.algo

'gbm'

In [57]:
dir(aml.leader)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bc',
 '_bcin',
 '_check_and_save_parm',
 '_check_targets',
 '_compute_algo',
 '_end_time',
 '_estimator_type',
 '_future',
 '_get_metrics',
 '_have_mojo',
 '_have_pojo',
 '_id',
 '_is_xvalidated',
 '_job',
 '_keyify_if_h2oframe',
 '_make_model',
 '_metrics_class',
 '_model_json',
 '_parms',
 '_plot',
 '_requires_training_frame',
 '_resolve_model',
 '_run_time',
 '_start_time',
 '_train',
 '_verify_training_frame_params',
 '_xval_keys',
 'actual_params',
 'aic',
 'algo',
 'auc',
 'balance_classes',
 'biases',
 'build_tree_one_node',
 'calibrate_model',
 'calibration_frame',
 'categorical_encoding',
 'cato

## Ensemble Exploration

In [50]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,GBM_3_AutoML_20190407_222223,10.393817,3.223944,10.393817,1.798379,0.025525
1,GBM_1_AutoML_20190407_222223,14.531027,3.811958,14.531027,2.399321,0.053515
2,DeepLearning_1_AutoML_20190407_222223,14.694030,3.833279,14.694030,2.779418,NaN
3,XGBoost_3_AutoML_20190407_222223,16.511499,4.063434,16.511499,2.657589,0.068602
4,GBM_2_AutoML_20190407_222223,17.863356,4.226506,17.863356,2.613506,0.057406
5,XGBoost_1_AutoML_20190408_000952,19.213204,4.383287,19.213204,2.589897,0.049718
6,StackedEnsemble_AllModels_AutoML_20190407_222223,21.622982,4.650052,21.622982,3.149146,0.185550
7,XGBoost_3_AutoML_20190408_001901,25.385364,5.038389,25.385364,3.308930,0.096640
8,XGBoost_2_AutoML_20190408_000952,25.893189,5.088535,25.893189,2.886280,0.052487
9,XGBoost_1_AutoML_20190407_222223,28.412796,5.330365,28.412796,3.359290,0.099527


## Getting Models

In [53]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[1])

In [52]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'XGBoost_1_AutoML_20190408_000952',
   'type': 'Key<Model>',
   'URL': '/3/Models/XGBoost_1_AutoML_20190408_000952'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour1.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour1.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'fold_assignment': {'default': 'AUTO', 'actual': 'Modulo'},
 'fold_column': {'def

# Model with runtime 1000 seconds

In [10]:
# Assume the following are passed by the user from the web interface

'''
Need a user id and project id?

'''
target='cnt' 
data_file='hour.csv'
run_time=1000
run_id='SOME_ID_20180617_221529' # Just some arbitrary ID
server_path='Users/newzysharma/Desktop/Desktop/Machine_Learning/Project'
classification=True
scale=False
max_models=None
balance_y=False # balance_classes=balance_y
balance_threshold=0.2
project ="HyperparameterDB_Project"  # project_name = project

In [22]:
# Use local data file or download from some type of bucket
import os

data_path=os.path.join(server_path,data_file)
data_path

'Users/newzysharma/Desktop/Desktop/Machine_Learning/Project/hour.csv'

In [11]:
# assign target and inputs for logistic regression
y = target
X = [name for name in df.columns if name != y]
print(y)
print(X)

cnt
['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered']


In [18]:
# determine column types
ints, reals, enums = [], [], []
for key, val in df.types.items():
    if key in X:
        if val == 'enum':
            enums.append(key)
        elif val == 'int':
            ints.append(key)            
        else: 
            reals.append(key)

print(ints)
print(enums)
print(reals)

['instant', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'casual', 'registered']
[]
['dteday', 'temp', 'atemp', 'hum', 'windspeed']


In [19]:
# impute missing values
_ = df[reals].impute(method='mean')
_ = df[ints].impute(method='median')

if scale:
    df[reals] = df[reals].scale()
    df[ints] = df[ints].scale()

In [20]:
# set target to factor for classification by default or if user specifies classification
if classification:
    df[y] = df[y].asfactor()

In [21]:
df[y].levels()

[['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '79',
  '80',
  '81',
  '82',
  '83',
  '84',
  '85',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '100',
  '101',
  '102',
  '103',
  '104',
  '105',
  '106',
  '107',
  '108',
  '109',
  '110',
  '111',
  '112',
  '113',
  '114',
  '115',
  '116',
  '117',
  '118',
  '119',
  '120',
  '121',
  '122',
  '123',
 

In [23]:
if classification:
    class_percentage = y_balance=df[y].mean()[0]/(df[y].max()-df[y].min())
    if class_percentage < balance_threshold:
        balance_y=True
        

print(run_time)
type(run_time)

1000


int


## Cross-validate rather than take a test training split with 1000 seconds

In [24]:
# automl
# runs for run_time seconds then builds a stacked ensemble

aml = H2OAutoML(max_runtime_secs=run_time,project_name = project) # init automl, run for 1000 seconds
aml.train(x=X,  
           y=y,
           training_frame=hp)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Leaderboard

In [25]:
# view leaderboard
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_3_AutoML_20190407_222223,10.3938,3.22394,10.3938,1.79838,0.0255248
GBM_1_AutoML_20190407_222223,14.531,3.81196,14.531,2.39932,0.0535151
DeepLearning_1_AutoML_20190407_222223,14.694,3.83328,14.694,2.77942,nan
XGBoost_3_AutoML_20190407_222223,16.5115,4.06343,16.5115,2.65759,0.0686023
GBM_2_AutoML_20190407_222223,17.8634,4.22651,17.8634,2.61351,0.0574055
StackedEnsemble_AllModels_AutoML_20190407_222223,21.623,4.65005,21.623,3.14915,0.18555
XGBoost_1_AutoML_20190407_222223,28.4128,5.33037,28.4128,3.35929,0.0995271
XGBoost_2_AutoML_20190407_222223,29.9311,5.47093,29.9311,3.50427,nan
GBM_4_AutoML_20190407_222223,33.269,5.76793,33.269,3.79358,nan
GLM_grid_1_AutoML_20190407_222223_model_1,38.387,6.19573,38.387,4.45197,nan


In [26]:
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_3_AutoML_20190407_222223


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 2.155227478290276
RMSE: 1.4680693029589154
MAE: 0.9893247655273629
RMSLE: 0.01894810024709767
Mean Residual Deviance: 2.155227478290276

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 10.39381683106471
RMSE: 3.2239442971404935
MAE: 1.798379228502425
RMSLE: 0.025524841564696307
Mean Residual Deviance: 10.39381683106471
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,1.7983793,0.0308816,1.8304428,1.7734675,1.7317806,1.8571346,1.7990708
mean_residual_deviance,10.393801,0.5789512,9.861467,9.708306,10.298363,11.979713,10.121156
mse,10.393801,0.5789512,9.861467,9.708306,10.298363,11.979713,10.121156
r2,0.9996840,0.0000176,0.9997007,0.9997072,0.9996852,0.9996366,0.9996907
residual_deviance,10.393801,0.5789512,9.861467,9.708306,10.298363,11.979713,10.121156
rmse,3.2215536,0.0877322,3.1402972,3.1158154,3.2091062,3.4611723,3.1813765
rmsle,0.0254593,0.0012937,0.0273739,0.0244253,0.0229992,0.0247157,0.0277825


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-07 22:38:06,13.517 sec,0.0,181.3823804,142.3998489,32899.5679309
,2019-04-07 22:38:06,13.582 sec,5.0,107.6288213,84.4261234,11583.9631809
,2019-04-07 22:38:06,13.653 sec,10.0,63.8797153,50.0275595,4080.6180310
,2019-04-07 22:38:06,13.722 sec,15.0,38.3081093,29.9323746,1467.5112348
,2019-04-07 22:38:06,13.791 sec,20.0,22.8327372,17.7868675,521.3338901
---,---,---,---,---,---,---
,2019-04-07 22:38:08,15.781 sec,185.0,1.5150515,1.0157186,2.2953812
,2019-04-07 22:38:08,15.834 sec,190.0,1.5003954,1.0083877,2.2511862
,2019-04-07 22:38:08,15.886 sec,195.0,1.4894064,1.0026039,2.2183313
,2019-04-07 22:38:08,15.938 sec,200.0,1.4792240,0.9959397,2.1881035



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,2522182656.0000000,1.0,0.9019286
casual,173936336.0000000,0.0689626,0.0621994
hr,53307732.0000000,0.0211356,0.0190628
workingday,22751658.0000000,0.0090206,0.0081360
temp,10054809.0000000,0.0039866,0.0035956
dteday,6581487.5,0.0026094,0.0023535
atemp,1724350.3750000,0.0006837,0.0006166
weekday,1579544.0,0.0006263,0.0005648
weathersit,1216864.2500000,0.0004825,0.0004351
instant,1135067.1250000,0.0004500,0.0004059


In [58]:
aml.leader.algo

'gbm'

In [59]:
dir(aml.leader)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bc',
 '_bcin',
 '_check_and_save_parm',
 '_check_targets',
 '_compute_algo',
 '_end_time',
 '_estimator_type',
 '_future',
 '_get_metrics',
 '_have_mojo',
 '_have_pojo',
 '_id',
 '_is_xvalidated',
 '_job',
 '_keyify_if_h2oframe',
 '_make_model',
 '_metrics_class',
 '_model_json',
 '_parms',
 '_plot',
 '_requires_training_frame',
 '_resolve_model',
 '_run_time',
 '_start_time',
 '_train',
 '_verify_training_frame_params',
 '_xval_keys',
 'actual_params',
 'aic',
 'algo',
 'auc',
 'balance_classes',
 'biases',
 'build_tree_one_node',
 'calibrate_model',
 'calibration_frame',
 'categorical_encoding',
 'cato

## Ensemble Exploration

In [27]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,GBM_3_AutoML_20190407_222223,10.393817,3.223944,10.393817,1.798379,0.025525
1,GBM_1_AutoML_20190407_222223,14.531027,3.811958,14.531027,2.399321,0.053515
2,DeepLearning_1_AutoML_20190407_222223,14.694030,3.833279,14.694030,2.779418,NaN
3,XGBoost_3_AutoML_20190407_222223,16.511499,4.063434,16.511499,2.657589,0.068602
4,GBM_2_AutoML_20190407_222223,17.863356,4.226506,17.863356,2.613506,0.057406
5,StackedEnsemble_AllModels_AutoML_20190407_222223,21.622982,4.650052,21.622982,3.149146,0.185550
6,XGBoost_1_AutoML_20190407_222223,28.412796,5.330365,28.412796,3.359290,0.099527
7,XGBoost_2_AutoML_20190407_222223,29.931125,5.470935,29.931125,3.504274,NaN
8,GBM_4_AutoML_20190407_222223,33.269049,5.767933,33.269049,3.793576,NaN
9,GLM_grid_1_AutoML_20190407_222223_model_1,38.387049,6.195728,38.387049,4.451969,NaN


##### As per the rmse metric that "The smaller the RMSE value, the better the model".
##### As per the mse metric that "The smaller the MSE value, the better the model".
So, In our case GBM_3_AutoML_20190407_222223 has the smallest rmse = 3.223944 and mse = 10.393817 that is why it is our best model.

In [33]:
aml_leaderboard_df.head()

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,GBM_3_AutoML_20190407_222223,10.393817,3.223944,10.393817,1.798379,0.025525
1,GBM_1_AutoML_20190407_222223,14.531027,3.811958,14.531027,2.399321,0.053515
2,DeepLearning_1_AutoML_20190407_222223,14.694030,3.833279,14.694030,2.779418,NaN
3,XGBoost_3_AutoML_20190407_222223,16.511499,4.063434,16.511499,2.657589,0.068602
4,GBM_2_AutoML_20190407_222223,17.863356,4.226506,17.863356,2.613506,0.057406


## Getting Models

In [35]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[5])

In [31]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_1_AutoML_20190407_222223',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_1_AutoML_20190407_222223'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_hour1.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_hour1.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 5},
 'fold_assignment': {'default': 'AUTO